In [ ]:
import pandas as pd
import numpy as np
import re
import subprocess
from datetime import timedelta
from tqdm import tqdm

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split

from sklearn import metrics
from scipy import stats

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from config.config import SQLQuery
q = SQLQuery('snowflake')

#### extracting response base from 29 Nov - 5 June

In [ ]:
resp_base = q(r"""
with lending_businesses as (
select distinct case when json_extract_path_text(predict_meta,'business_id') is null 
     then json_extract_path_text(json_extract_path_text(predict_meta,'data') , 'business_id')
     else json_extract_path_text(predict_meta,'business_id') end as business_id,
-- lending_business_id,
date(created_at) as decision_date
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS
where created_at >= '2022-11-29'
and created_at < '2023-06-06'
),
target_businesses as (
SELECT DISTINCT a.id AS business_id, APPLICATION_ID, decision_date,
lower(regexp_replace(a.company_name, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '')) AS company_name, 
CASE WHEN dba IS NOT NULL THEN lower(regexp_replace(dba, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '')) ELSE NULL end AS dba,
case when a.TYPE = 'sole_proprietorship' then 'sol_prop' else 'non sol_prop' end AS business_type
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC.BUSINESSES a 
join prod_db.data.businesses b
on a.id = b.business_id
join lending_businesses l
on l.business_id = a.id
),
logic1 as (
-- account name match
SELECT distinct t1.id as plaid_item_id, t1.business_id, json_extract_path_text(json_extract_path_text(t2.value,'account'),'name') as ext_account_name,
lower(regexp_replace(json_extract_path_text(json_extract_path_text(t2.value,'account'),'name'), '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '')) AS external_acc_company_name ,
json_extract_path_text(json_extract_path_text(t2.value,'ach'),'account') AS account_id
FROM FIVETRAN_DB.plaid_data_public.plaid_items t1, LATERAL FLATTEN(input => "META") t2
-- WHERE status = 'active'
where account_id is not null
and account_id != ''
union
SELECT distinct t1.id as plaid_item_id, t1.business_id, t2.value as ext_account_name,
lower(regexp_replace(t2.value, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '')) AS external_acc_company_name, 
--'' as account_id,
json_extract_path_text(json_extract_path_text(t3.value,'ach'),'account') AS account_id
FROM FIVETRAN_DB.plaid_data_public.plaid_items t1, LATERAL FLATTEN(input => "IDENTITY":names) t2
, LATERAL FLATTEN(input => "META") t3
-- WHERE status = 'active'
where account_id is not null
and account_id != ''
order by account_id
),
mapped_accts as (
select distinct a.*, b.plaid_item_id, b.ext_account_name, b.external_acc_company_name, b.account_id, e.owned_by,
case when ((company_name = external_acc_company_name) OR 
(SOUNDEX(company_name) = SOUNDEX(external_acc_company_name)) OR
(dba IS NOT NULL AND dba = external_acc_company_name) OR
(dba IS NOT NULL AND SOUNDEX(dba)=SOUNDEX(external_acc_company_name))) 
or ((len(external_acc_company_name)>3) and ((company_name LIKE '%' || external_acc_company_name || '%') or (external_acc_company_name LIKE '%' || company_name || '%')))
or ((len(external_acc_company_name)>3) and (len(dba)>3) and (dba IS NOT NULL) and ((dba LIKE '%' || external_acc_company_name || '%') or (external_acc_company_name LIKE '%' || dba || '%')))
then 1 else 0 end as name_match_flag
from target_businesses a
left join logic1 b 
on a.business_id = b.business_id
left join metabase_db.external.external_accounts e
on a.business_id = e.business_id
and b.account_id = e.account_number
where account_id is not null
and account_id !=''
), --6.78%
logic2 as (
-- email id match
select distinct a.business_id, a.company_name, a.dba, email, email_domain, website
from target_businesses a
left join PROD_DB.DATA.APPLICATIONS b
on a.application_id = b.application_id
),
plaid_email as (
select distinct business_id, t1.id as plaid_item_id, json_extract_path_text(t2.value,'data') as email,
json_extract_path_text(json_extract_path_text(t3.value,'ach'),'account') AS account_id
from fivetran_db.plaid_data_public.plaid_items t1, LATERAL FLATTEN(input => "IDENTITY":emails) t2
, LATERAL FLATTEN(input => "META") t3
-- where status = 'active'
),
match_email as (
select b.*, plaid_item_id, lower(t1.email) as plaid_email, account_id,
case when (email_domain not in ('gmail.com','yahoo.com','outlook.com','hotmail.com','icloud.com','protonmail.com',
'me.com','live.com','mail.com','comcast.net','att.net','msn.com','pm.me','mac.com') and 
          (
          soundex(b.email_domain) = soundex(split_part(plaid_email,'@',2)) 
          or b.email_domain = split_part(plaid_email,'@',2) 
          
          or soundex(company_name) = soundex(split_part(split_part(plaid_email,'@',2),'.',1)) 
          or company_name = split_part(split_part(plaid_email,'@',2),'.',1)
          
          or soundex(dba) = soundex(split_part(split_part(plaid_email,'@',2),'.',1)) 
          or dba = split_part(split_part(plaid_email,'@',2),'.',1)))
          
          or b.email = plaid_email
          
          or split_part(b.email_domain,'.',1) = split_part(plaid_email,'@',1)
          
          
          or ((len(split_part(plaid_email,'@',1))>3) and (len(dba)>3) and (plaid_email IS NOT NULL) and ((dba LIKE '%' || split_part(plaid_email,'@',1) || '%') or (split_part(plaid_email,'@',1) LIKE '%' || dba || '%')))
          or ((len(split_part(split_part(plaid_email,'@',2),'.',1))>3) and (len(dba)>3) and (plaid_email IS NOT NULL) and ((dba LIKE '%' || split_part(split_part(plaid_email,'@',2),'.',1) || '%') or split_part(split_part(plaid_email,'@',2),'.',1) LIKE '%' || dba || '%'))
          
          or ((len(split_part(plaid_email,'@',1))>3) and (len(company_name)>3) and (plaid_email IS NOT NULL) and ((company_name LIKE '%' || split_part(plaid_email,'@',1) || '%') or (split_part(plaid_email,'@',1) LIKE '%' || company_name || '%')))
          or ((len(split_part(split_part(plaid_email,'@',2),'.',1))>3) and (len(company_name)>3) and (plaid_email IS NOT NULL) and ((company_name LIKE '%' || split_part(split_part(plaid_email,'@',2),'.',1) || '%') or split_part(split_part(plaid_email,'@',2),'.',1) LIKE '%' || company_name || '%'))
          
          then 1 else 0 end as email_match_flag
from logic2 b
left join plaid_email t1
on t1.business_id = b.business_id
order by business_id
), 
logic4 as (
--website match
select distinct b.business_id, p.plaid_item_id, p.account_id, a.external_acc_company_name, lower(split_part(replace(b.website,'www.','.'),'.',1)) as cleaned_website, 
split_part(split_part(replace(lower(b.website),'www.','.'),'.',1),'//',2) as cleaned_website2, 
case when (b.website is not null) and (soundex(external_acc_company_name) = soundex(cleaned_website) 
          or (external_acc_company_name = cleaned_website)
          or (cleaned_website = split_part(split_part(p.email,'@',2),'.',1))
          or (cleaned_website = split_part(p.email,'@',1))
          or (soundex(cleaned_website) = soundex(split_part(split_part(p.email,'@',2),'.',1))) 
          or (soundex(cleaned_website) = soundex(split_part(p.email,'@',1))) 
          or (soundex(external_acc_company_name) = soundex(cleaned_website2)) 
          or (external_acc_company_name = cleaned_website2)
          or (cleaned_website2 = split_part(split_part(p.email,'@',2),'.',1))
          or (cleaned_website2 = split_part(p.email,'@',1))
          or (soundex(cleaned_website2) = soundex(split_part(split_part(p.email,'@',2),'.',1))) 
          or (soundex(cleaned_website2) = soundex(split_part(p.email,'@',1)))
          
          or ((len(cleaned_website)>3) and (len(split_part(split_part(p.email,'@',2),'.',1))>3) and ((split_part(split_part(p.email,'@',2),'.',1) LIKE '%' || cleaned_website || '%') or (cleaned_website LIKE '%' || split_part(split_part(p.email,'@',2),'.',1) || '%')))
          or ((len(cleaned_website2)>3) and (len(split_part(p.email,'@',1))>3) and ((split_part(split_part(p.email,'@',2),'.',1) LIKE '%' || cleaned_website2 || '%') or (cleaned_website2 LIKE '%' || split_part(p.email,'@',1) || '%')))
          
          or ((len(cleaned_website)>3) and (len(split_part(split_part(p.email,'@',2),'.',1))>3) and ((split_part(split_part(p.email,'@',2),'.',1) LIKE '%' || cleaned_website || '%') or (cleaned_website LIKE '%' || split_part(split_part(p.email,'@',2),'.',1) || '%')))
          or ((len(cleaned_website2)>3) and (len(split_part(p.email,'@',1))>3) and ((split_part(p.email,'@',1) LIKE '%' || cleaned_website2 || '%') or (cleaned_website2 LIKE '%' || split_part(p.email,'@',1) || '%')))
          
          or ((len(cleaned_website)>3) and (len(external_acc_company_name)>3) and ((external_acc_company_name LIKE '%' || cleaned_website || '%') or (cleaned_website LIKE '%' || external_acc_company_name || '%')))
          or ((len(cleaned_website2)>3) and (len(external_acc_company_name)>3) and ((external_acc_company_name LIKE '%' || cleaned_website2 || '%') or (cleaned_website2 LIKE '%' || external_acc_company_name || '%')))
          
          )
          
          then 1 else 0 end as website_match
from logic2 b
left join plaid_email p
on b.business_id = p.business_id
left join logic1 a
on a.business_id = b.business_id
and p.account_id = a.account_id
),
final as (
select distinct s.business_id, s.business_type, s.decision_date, a.company_name, a.dba, a.external_acc_company_name,
case when a.plaid_item_id is null and d.plaid_item_id is null then b.plaid_item_id
     when a.plaid_item_id is null and b.plaid_item_id is null then d.plaid_item_id
     else a.plaid_item_id end as plaid_item_id, 
case when a.account_id is null and d.account_id is null then b.account_id
     when a.account_id is null and b.account_id is null then d.account_id
     else a.account_id end as account_id, 
a.owned_by,
b.email, b.plaid_email,cleaned_website,
max(a.name_match_flag) as name_match_flag,
max(b.email_match_flag) as email_match_flag,
max(website_match) as website_match_flag
from target_businesses s
left join mapped_accts a
on s.business_id = a.business_id
left join match_email b
on a.business_id = b.business_id
and a.account_id = b.account_id
left join logic4 d
on d.business_id = s.business_id
and a.account_id = d.account_id
group by 1,2,3,4,5,6,7,8,9,10,11,12
),
external_accts as (
select *,
case when name_match_flag=1 or email_match_flag=1 or website_match_flag=1
then 1 else 0 end as final_flag
from final
),
latest_mapped_acct as (
select * from (
    select distinct e.business_id, e.account_number, e.id, e.created_at,
    row_number() over(partition by e.business_id, e.account_number order by e.created_at desc) as row_num
    from METABASE_DB.EXTERNAL.EXTERNAL_ACCOUNTS e
    join lending_businesses l
    on e.business_id = l.business_id)
where row_num =1
),
base1 as (
select distinct a.business_id, a.decision_date, a.account_id, d.id as EXTERNAL_ACCOUNT_ID, final_flag as logic_match_flag
-- case when business_type='sol_prop' --and (a.owned_by is null or a.owned_by='' or a.owned_by='business') )
-- or business_type!='sol_prop' then 1 else 0 end as business_acct_flag
from external_accts a
left join latest_mapped_acct d
on a.account_id = d.account_number
and a.business_id = d.business_id
and a.account_id is not null
and a.account_id!=''
order by business_id, account_id
),
initial_match as (
select *
from base1
where logic_match_flag = 1 
-- and business_acct_flag = 1
),
min_txns as (
select t.business_id, min(date) as min_txn_date
FROM FIVETRAN_DB.plaid_data_public.PLAID_TRANSACTIONS t
join (select distinct business_id, decision_date, EXTERNAL_ACCOUNT_ID 
from initial_match) b
on t.business_id = b.business_id 
and date(t.date) >= date(dateadd(day,-360,decision_date))  
and date(t.date) < date(decision_date)
and t.external_account_id = b.external_account_id
where pending = FALSE 
and (_fivetran_deleted = FALSE or _fivetran_deleted is null)
and t.EXTERNAL_ACCOUNT_ID is not null
group by 1
),
ext_txns as (
select distinct b.business_id, b.decision_date, t.EXTERNAL_ACCOUNT_ID,
date(date) as date, amount, TRANSACTION_id, type, description
FROM FIVETRAN_DB.plaid_data_public.PLAID_TRANSACTIONS t
join (select distinct business_id, decision_date, EXTERNAL_ACCOUNT_ID 
from initial_match) b
on t.business_id = b.business_id
and date(t.date) >= date(dateadd(day,-180,decision_date))  
and date(t.date) < date(decision_date)
and t.external_account_id = b.external_account_id
where pending = FALSE 
and (_fivetran_deleted = FALSE or _fivetran_deleted is null
and t.EXTERNAL_ACCOUNT_ID is not null)
),
txns_count as (
select business_id, count(distinct TRANSACTION_id) as count_txns
from ext_txns
group by 1
),
matched_businesses as (
select * from (
select distinct a.business_id,
a.decision_date,
date(dateadd(day,-180,decision_date)) as txn_start_date,
a.account_id,
a.external_account_id,
a.logic_match_flag,
-- a.business_acct_flag,
case when (min_txn_date < date(dateadd(day,-150,decision_date))) then 1 else 0 end as sixth_mnth_txn_avail_flag,
case when (count_txns > 0) then 1 else 0 end as txn_avail_flag,
case when external_account_id is null then 0 else 1 end as ext_acct_avail_flag
from base1 a
left join txns_count b
on a.business_id = b.business_id
left join min_txns c
on a.business_id = c.business_id)
where logic_match_flag = 1 
-- and business_acct_flag = 1
and sixth_mnth_txn_avail_flag = 1
and txn_avail_flag = 1
and ext_acct_avail_flag = 1
)

select *
from matched_businesses 
""")
resp_base.shape, resp_base.business_id.nunique()

((3236, 9), 1384)

In [ ]:
resp_base.head()

,business_id,decision_date,txn_start_date,account_id,external_account_id,logic_match_flag,sixth_mnth_txn_avail_flag,txn_avail_flag,ext_acct_avail_flag
0,ef94d738-20f3-45c2-9c11-9a636e469885,2023-03-12,2022-09-13,1110025509491,5e224820-2a09-4b70-bcb6-5d17ff326091,1,1,1,1
1,a2317f88-325b-43ee-9423-6c711856555b,2023-02-02,2022-08-06,446037014555,58cecb8c-5a35-4109-8cc5-186a4c80c712,1,1,1,1
2,252c9a3f-338f-43fc-893f-87e4c8184e8c,2023-03-15,2022-09-16,193328941,a3924492-77d6-4a3e-a10d-f66c183f3730,1,1,1,1
3,252c9a3f-338f-43fc-893f-87e4c8184e8c,2023-03-15,2022-09-16,199586667,af8b6a29-a923-4477-a72f-11b230d33d46,1,1,1,1
4,ef94d738-20f3-45c2-9c11-9a636e469885,2023-03-12,2022-09-13,1110008414843,98c2a099-fb5f-48b8-a17d-1093d15a9f7f,1,1,1,1


In [ ]:
from sqlalchemy.types import NVARCHAR

resp_base.to_sql(name='lending_response_base_w_plaid_match_v2_20230623',
                 con=q.engine, 
                 schema='prod_db.adhoc',
                 if_exists='append', 
                 index=False, 
                 chunksize=16000, 
                 method='multi',
                 dtype={col_name: NVARCHAR for col_name in resp_base})

3236

#### extracting last 6m transactions for extracted base

In [ ]:
txns = q(r""" 
select distinct t.*
FROM FIVETRAN_DB.plaid_data_public.PLAID_TRANSACTIONS t
join prod_db.adhoc.lending_response_base_w_plaid_match_v2_20230623 b
on t.business_id = b.business_id
and t.EXTERNAL_ACCOUNT_ID = b.EXTERNAL_ACCOUNT_ID
and date(t.date) >= date(b.txn_start_date)
and date(t.date) < b.decision_date
where t.pending = FALSE 
and (t._fivetran_deleted = FALSE or t._fivetran_deleted is null)
""")
txns.shape, txns.business_id.nunique()

((504387, 20), 1384)

In [ ]:
txns.to_sql(name='lending_dda_l6m_plaid_txns_20230623',
                 con=q.engine, 
                 schema='prod_db.adhoc',
                 if_exists='append', 
                 index=False, 
                 chunksize=16000, 
                 method='multi',
                 dtype={col_name: NVARCHAR for col_name in txns})

504387

#### merging the txns with the approval base with target

In [ ]:
resp = q(r""" select * from prod_db.adhoc.lending_response_base_w_plaid_match_v2_20230623 """)

appr = q(r""" select * from prod_db.adhoc.LENDING_BASE_TARGET_23JUN """)

resp.shape, appr.shape

((3236, 9), (708, 7))

In [ ]:
appr.head(2)

,lending_business_id,business_id,created_at,drawn_flag,everDPD_15,fico_score,target
0,5249377a-500b-42a7-af3e-109b648c928d,b14a4191-4139-446d-aa66-b071e4eb8ccb,2023-03-24 17:29:54.080531+00:00,1,None,800,0
1,fe14006a-6546-43a3-a1a6-88c4993f8f3f,416d0695-5f0f-4d37-88d8-53ecad0b1f5f,2023-04-25 23:30:57.040311+00:00,1,0,600,0


In [ ]:
base = resp[['business_id','decision_date']].drop_duplicates().merge(appr,on=['business_id']).reset_index(drop=True)
base.shape

(708, 8)

In [ ]:
base = base[['business_id','lending_business_id','decision_date','drawn_flag','everDPD_15','fico_score','target']]
print(base.shape)
base.head()

(708, 7)


,business_id,lending_business_id,decision_date,drawn_flag,everDPD_15,fico_score,target
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,1,0,721,0
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,1,0,846,0
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,1,0,720,0
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,1,0,720,0
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,1,0,687,0


#### filtering txns base only for approvals

In [ ]:
txns = q(r""" 
select * 
from prod_db.adhoc.lending_dda_l6m_plaid_txns_20230623
where business_id in (
select distinct business_id from prod_db.adhoc.LENDING_BASE_TARGET_23JUN
)
""")
txns.shape, txns.business_id.nunique()

((242838, 20), 708)

In [ ]:
temp = txns.groupby('business_id')['transaction_id'].nunique().reset_index()
buss_id = temp[temp.transaction_id>20].business_id.unique().tolist()

base_f = base[base.business_id.isin(buss_id)].reset_index(drop=True)
base_f.shape, base_f.business_id.nunique()

((646, 7), 646)

In [ ]:
txns_f = txns[txns.business_id.isin(buss_id)].reset_index(drop=True)
txns_f.shape, txns_f.business_id.nunique()

((242282, 20), 646)

In [ ]:
base_f.head()

,business_id,lending_business_id,decision_date,drawn_flag,everDPD_15,fico_score,target
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,1,0,721,0
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,1,0,846,0
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,1,0,720,0
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,1,0,720,0
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,1,0,687,0


In [ ]:
txns_f.head()

,id,transaction_id,date,amount,plaid_account_id,pending,description,created_at,merchant_name,retrieved_at,type,plaid_item_id,external_account_id,updated_at,meta,iso_currency_code,categories,business_id,_fivetran_deleted,_fivetran_synced
0,c9d76863-62f2-495e-9fb3-05e7f94098b2,qeYbVB34YQu07Lo4rrdqfvE6qyJnRPCxaq3bq,2023-01-03,11.64,5KeXB1ZdevT8bmyLXXbKtekjgbq0EwTNL43Xj,false,QUICK CHEK FOO 12/31 #XXXXX5915 MOBILE PURCHAS...,2023-02-01 09:59:03.876000+00:00,QuickChek,2023-02-01 09:59:03.865000+00:00,debit,82c8cec7-ecf3-4f2f-a35e-bf2ea3390140,e82d9e16-fe88-473e-9fdb-b86c31c4ed72,2023-02-01 09:59:03.876000+00:00,"{\n ""account_id"": ""5KeXB1ZdevT8bmyLXXbKtekjgb...",USD,"Shops,Convenience Stores",24847a04-fdb1-411b-986a-67785732c800,None,2023-02-20 19:07:57.754000+00:00
1,781376a5-7437-4395-ae0d-fcca4c878c3c,nBPAA46qxLtLM1DaKb7rHrLR9AQO8JU0MrZKn,2022-08-29,6.96,vjvggVyZ5KUKMyEY8BV8iOYgbqBaABcm8zbOQ,false,Debit: Signature purchase from 435678000424939...,2022-10-07 02:48:22.883000+00:00,Love's,2022-10-07 02:48:22.883000+00:00,debit,74fbba95-3a5e-4969-bfb9-b8d2ceffd52e,01540901-b19e-4e13-aaa0-bcfc4a430cc1,2022-10-07 02:48:22.883000+00:00,"{\n ""account_id"": ""vjvggVyZ5KUKMyEY8BV8iOYgbq...",USD,"Travel,Gas Stations",fdbeb997-87d8-411e-a068-9a2297324910,None,2023-02-20 15:32:15.801000+00:00
2,e67f935c-813d-49d1-a8d8-65d17638f069,QKK5Z9YPKrijzMzPXeedIYvBpBrpZPiwEaep8,2023-01-17,100.00,9rrwN1y6rEFgB9Bn11agin1L3gkNxaFd7XZVm,false,SVCS42b21c 01-15 g.co/walleth# CA 2854 DEBIT C...,2023-01-19 08:28:49.787000+00:00,Google Play Store,2023-01-19 08:28:49.787000+00:00,debit,a923b118-7a37-4582-92cc-68f5daf9460b,7daba2a8-fb94-4abc-aaf8-65d2a7f1214a,2023-01-19 08:28:49.787000+00:00,"{\n ""account_id"": ""9rrwN1y6rEFgB9Bn11agin1L3g...",USD,"Shops,Digital Purchase",830bb334-3fba-4de7-9124-cb6de0cef4b3,None,2023-02-20 20:24:25.502000+00:00
3,7940c3c4-00c3-4bc5-8792-dba27e80fdef,YdBNDaKn1MhArx84PyBRTMQjYMXE6NCYRb6q7,2022-08-22,-40.00,0JK79B31aQTjMvg8bjKLf1XVnR3LDNSVPJyPN,false,Venmo,2023-01-29 06:19:44.857000+00:00,None,2023-01-29 06:19:44.856000+00:00,credit,2f8d1fcf-79c4-4eaa-b31c-04647cffa051,5e40ad0a-5b57-4b97-b61d-5889fa80e270,2023-01-29 06:19:44.857000+00:00,"{\n ""account_id"": ""0JK79B31aQTjMvg8bjKLf1XVnR...",USD,"Transfer,Third Party,Venmo",6ee06fa4-0720-41a4-853f-46eaf60c94e6,None,2023-02-20 15:34:30.892000+00:00
4,dfc839dd-68e5-4119-9053-19df7ff58b17,0MVob6BkdmtndgadNmzEc18Zz36g79tBZ9aZEv,2022-10-07,10.00,qk0PjONgBRCKVJkVOj03C1zmaeoaePFJeyRDy,false,PURCHASE AUTHORIZED ON 10/05 FPRAZ - KJZZ/K-BA...,2023-06-17 07:03:11.796000+00:00,Fpraz,2023-06-17 07:03:11.795000+00:00,debit,7fb87313-270c-4df8-b6d1-68796aa5f595,f3f06b4b-d645-4b09-a8d5-669656c1622f,2023-06-17 07:03:11.796000+00:00,"{\n ""account_id"": ""qk0PjONgBRCKVJkVOj03C1zmae...",USD,"Transfer,Debit",b14a4191-4139-446d-aa66-b071e4eb8ccb,None,2023-06-17 07:20:18.399000+00:00


In [ ]:
from sqlalchemy.types import NVARCHAR

base_f.to_sql(name='lending_response_base_w_plaid_match_g20_txns_20230624',
                 con=q.engine, 
                 schema='prod_db.adhoc',
                 if_exists='append', 
                 index=False, 
                 chunksize=16000, 
                 method='multi',
                 dtype={col_name: NVARCHAR for col_name in base_f})

646

In [ ]:
txns_f.to_sql(name='lending_dda_l6m_plaid_txns_g20_txns_20230624',
                 con=q.engine, 
                 schema='prod_db.adhoc',
                 if_exists='append', 
                 index=False, 
                 chunksize=16000, 
                 method='multi',
                 dtype={col_name: NVARCHAR for col_name in txns})

242282

#### adding owned_by flag in txn data to filter the txns

In [ ]:
tagged_txns = q(r""" 
select a.*, e.owned_by
from prod_db.adhoc.lending_dda_l6m_plaid_txns_g20_txns_20230624 a
left join METABASE_DB.EXTERNAL.EXTERNAL_ACCOUNTS e
on a.external_account_id = e.id
and a.business_id = e.business_id
""")
tagged_txns.shape, tagged_txns.business_id.nunique()

((242282, 21), 646)

In [ ]:
from sqlalchemy.types import NVARCHAR

tagged_txns.to_sql(name='lending_dda_txns_w_owned_20230624',
                 con=q.engine, 
                 schema='prod_db.adhoc',
                 if_exists='append', 
                 index=False, 
                 chunksize=16000, 
                 method='multi',
                 dtype={col_name: NVARCHAR for col_name in tagged_txns})

242282

#### filtering txns logic (basis owned_by and txn type)

In [ ]:
filtered_txns = q(r""" 
with txn_base as (
select * 
from prod_db.adhoc.lending_dda_txns_w_owned_20230624
where business_id in (
select distinct business_id from prod_db.adhoc.LENDING_BASE_TARGET_23JUN)
),
ext_acct_buss as
(select distinct external_account_id, 'business' as business_flag from 
(SELECT external_account_id, --business_id,
       CASE WHEN SUM(CASE WHEN type='debit' and OWNED_BY ='person' and
                       (categories like '%Transfer,Payroll%' or UPPER(DESCRIPTION)
                        like any ('%%PAYROLL%%', '%%ADP (AUTOMATIC DATA PROCESSING)%%', '%%PAYCHEX%%',
                                  '%%GUSTO%%', '%%SQUARE PAYROLL%%', '%%INTUIT PAYROLL (QUICKBOOKS PAYROLL)%%', '%%ONPAY%%', '%%SUREPAYROLL%%',
                                  '%%PATRIOT SOFTWARE%%',  '%%ZENEFITS%%', '%%JUSTWORKS%%',
                                  '%%WAGEPOINT%%', '%%ZENPAYROLL%%', '%%GUSTO%%',
                                  '%%FUSE WORKFORCE MANAGEMENT%%', '%%PAYCOR%%', '%%APS PAYROLL%%'))
                      THEN 1 ELSE 0 END) >= 1 AND
       SUM(CASE WHEN type='debit' and OWNED_BY ='person' and
                     (categories like '%Transfer,Payroll%' or UPPER(DESCRIPTION)
                        like any ('%%PAYROLL%%', '%%ADP (AUTOMATIC DATA PROCESSING)%%', '%%PAYCHEX%%',
                                  '%%GUSTO%%', '%%SQUARE PAYROLL%%', '%%INTUIT PAYROLL (QUICKBOOKS PAYROLL)%%', '%%ONPAY%%', '%%SUREPAYROLL%%',
                                  '%%PATRIOT SOFTWARE%%',  '%%ZENEFITS%%', '%%JUSTWORKS%%',
                                  '%%WAGEPOINT%%', '%%ZENPAYROLL%%', '%%GUSTO%%',
                                  '%%FUSE WORKFORCE MANAGEMENT%%', '%%PAYCOR%%', '%%APS PAYROLL%%')) THEN abs(amount) ELSE 0 END) > 1000
       THEN 1 ELSE 0 END AS flag_payroll,
       CASE WHEN SUM(CASE WHEN type ='credit' and OWNED_BY ='person' and UPPER(DESCRIPTION) ilike any ('%%RENT%%','%%COZY%%', '%%RENTPAYMENT%%', '%%ZILLOW RENTAL MANAGER%%',
                                                        '%%AVAIL%%', '%%RENTEC DIRECT%%', '%%BUILDIUM%%', '%%APPFOLIO%%',
                                                        '%%RENTLER%%', '%%TENANTCLOUD%%', '%%RENTREDI%%', '%%RENTMOOLA%%',
                                                        '%%RENTIGO%%', '%%RENTTRACK%%', '%%RENTPOST%%', '%%RENTMONITOR%%',
                                                        '%%RENTMERCHANT%%', '%%RENTEC PM%%', '%%RENTEC PAY%%',
                                                        '%%PAYYOURRENT%%', '%%RENTPAIDONLINE%%')
                      THEN 1 ELSE 0 END) >= 2 AND
       SUM(CASE WHEN type ='credit' and OWNED_BY ='person' and UPPER(DESCRIPTION) ilike any ('%%RENT%%','%%COZY%%', '%%RENTPAYMENT%%', '%%ZILLOW RENTAL MANAGER%%',
                                                        '%%AVAIL%%', '%%RENTEC DIRECT%%', '%%BUILDIUM%%', '%%APPFOLIO%%',
                                                        '%%RENTLER%%', '%%TENANTCLOUD%%', '%%RENTREDI%%', '%%RENTMOOLA%%',
                                                        '%%RENTIGO%%', '%%RENTTRACK%%', '%%RENTPOST%%', '%%RENTMONITOR%%',
                                                        '%%RENTMERCHANT%%', '%%RENTEC PM%%', '%%RENTEC PAY%%',
                                                        '%%PAYYOURRENT%%', '%%RENTPAIDONLINE%%') THEN abs(amount) ELSE 0 END) > 1000
       THEN 1 ELSE 0 END AS  flag_rent,
       CASE WHEN SUM(CASE WHEN type='credit' and OWNED_BY ='person' and UPPER(DESCRIPTION) ilike any ('%%AMAZON%%', '%%EBAY%%', '%%ETSY%%', '%%SHOPIFY%%', '%%WALMART MARKETPLACE%%',
                                                      '%%WOOCOMMERCE%%', '%%BIGCOMMERCE%%',
                                                      '%%SQUARE ONLINE STORE%%', '%%MAGENTO%%', '%%WIX STORES%%',
                                                      '%%SQUARESPACE COMMERCE%%',
                                                      '%%FACEBOOK MARKETPLACE%%', '%%INSTAGRAM SHOPPING%%',
                                                      '%%GOOGLE SHOPPING%%', '%%LINKEDIN MARKETPLACE%%', '%%PINTEREST SHOP%%',
                                                      '%%HOUZZ%%', '%%NEWEGG%%',
                                                      '%%RAKUTEN%%', '%%BONANZA%%', '%%ZIBBET%%', '%%HANDMADE AT AMAZON%%',
                                                      '%%STORENVY%%', '%%ECWID%%',
                                                      '%%WEEBLY ECOMMERCE%%', '%%CRATEJOY%%', '%%TICTAIL%%', '%%DEPOP%%', '%%RUBY LANE%%', '%%TOPHATTER%%',
                                                      '%%1STDIBS%%', '%%ARTFIRE%%', '%%ECRATER%%', '%%SOCIETY6%%', '%%REVERB%%',
                                                      '%%OFFERUP%%', '%%LETGO%%', '%%MERCARI%%', '%%POSHMARK%%', '%%GRAILED%%')
                      THEN 1 ELSE 0 END) >= 2 AND
       SUM(CASE WHEN type='credit' and OWNED_BY ='person' and UPPER(DESCRIPTION) ilike any ('%%AMAZON%%', '%%EBAY%%', '%%ETSY%%', '%%SHOPIFY%%', '%%WALMART MARKETPLACE%%',
                                                      '%%WOOCOMMERCE%%', '%%BIGCOMMERCE%%',
                                                      '%%SQUARE ONLINE STORE%%', '%%MAGENTO%%', '%%WIX STORES%%',
                                                      '%%SQUARESPACE COMMERCE%%',
                                                      '%%FACEBOOK MARKETPLACE%%', '%%INSTAGRAM SHOPPING%%',
                                                      '%%GOOGLE SHOPPING%%', '%%LINKEDIN MARKETPLACE%%', '%%PINTEREST SHOP%%',
                                                      '%%HOUZZ%%', '%%NEWEGG%%',
                                                      '%%RAKUTEN%%', '%%BONANZA%%', '%%ZIBBET%%', '%%HANDMADE AT AMAZON%%',
                                                      '%%STORENVY%%', '%%ECWID%%',
                                                      '%%WEEBLY ECOMMERCE%%', '%%CRATEJOY%%', '%%TICTAIL%%', '%%DEPOP%%', '%%RUBY LANE%%', '%%TOPHATTER%%',
                                                      '%%1STDIBS%%', '%%ARTFIRE%%', '%%ECRATER%%', '%%SOCIETY6%%', '%%REVERB%%',
                                                      '%%OFFERUP%%', '%%LETGO%%', '%%MERCARI%%', '%%POSHMARK%%', '%%GRAILED%%')  THEN abs(amount) ELSE 0 END) > 500
       THEN 1 ELSE 0 END AS  online_platform_deposit,
        CASE WHEN SUM(CASE WHEN type='credit' and
            OWNED_BY ='person' and UPPER(DESCRIPTION) ilike any ('%%SQUARE%%','%%POS%%', '%%CARD PAYMENT','%%STRIPE%%')
                      THEN 1 ELSE 0 END) >= 2 AND
             SUM(CASE WHEN type='credit' and OWNED_BY ='person' and
                 UPPER(DESCRIPTION) ilike any ('%%SQUARE%%','%%POS%%', '%%CARD PAYMENT','%%STRIPE%%')   THEN abs(amount) ELSE 0 END) > 1000
       THEN 1 ELSE 0 END AS  business_like_payment,
        CASE WHEN SUM(CASE WHEN OWNED_BY ='person' and UPPER(DESCRIPTION) ilike any
  ('%%ONDECK%%', '%%KABBAGE%%', '%%BLUEVINE%%', '%%FUNDBOX%%', '%%FUNDERA%%',
  '%%LENDIO%%', '%%CREDIBLY%%', '%%ACCION%%', '%%FUNDING CIRCLE%%', '%%STREETSHARES%%',
  '%%SMARTBIZ LOANS%%', '%%NATIONAL FUNDING%%', '%%BIZ2CREDIT%%', '%%CAN CAPITAL%%',
  '%%IOU FINANCIAL%%', '%%PAYPAL WORKING CAPITAL%%', '%%LENDINGCLUB%%', '%%QUARTERSPOT%%',
  '%%FORA FINANCIAL%%', '%%BOND STREET%%', '%%RAPID FINANCE%%', '%%HEADWAY CAPITAL%%',
  '%%SNAPCAP%%', '%%FORWARDLINE%%', '%%SHIELD FUNDING%%', '%%BFS CAPITAL%%', '%%MULLIGAN FUNDING%%',
  '%%CYOGATE%%', '%%CAPIFY%%', '%%EXCEL CAPITAL MANAGEMENT%%', '%%BALBOA CAPITAL%%', '%%CREST CAPITAL%%',
  '%%KEYSTONE FUNDING%%', '%%FAASTRAK%%', '%%HALO CAPITAL%%',
  '%%SNAP ADVANCES%%', '%%JET CAPITAL%%', '%%SHARPSHOOTER FUNDING%%', '%%MULLIGAN FUNDING%%', '%%GREENBOX CAPITAL%%')
                      THEN 1 ELSE 0 END) >0 AND
             SUM(CASE WHEN OWNED_BY ='person' and UPPER(DESCRIPTION) ilike any
  ('%%ONDECK%%', '%%KABBAGE%%', '%%BLUEVINE%%', '%%FUNDBOX%%', '%%FUNDERA%%',
  '%%LENDIO%%', '%%CREDIBLY%%', '%%ACCION%%', '%%FUNDING CIRCLE%%', '%%STREETSHARES%%',
  '%%SMARTBIZ LOANS%%', '%%NATIONAL FUNDING%%', '%%BIZ2CREDIT%%', '%%CAN CAPITAL%%',
  '%%IOU FINANCIAL%%', '%%PAYPAL WORKING CAPITAL%%', '%%LENDINGCLUB%%', '%%QUARTERSPOT%%',
  '%%FORA FINANCIAL%%', '%%BOND STREET%%', '%%RAPID FINANCE%%', '%%HEADWAY CAPITAL%%',
  '%%SNAPCAP%%', '%%FORWARDLINE%%', '%%SHIELD FUNDING%%', '%%BFS CAPITAL%%', '%%MULLIGAN FUNDING%%',
  '%%CYOGATE%%', '%%CAPIFY%%', '%%EXCEL CAPITAL MANAGEMENT%%', '%%BALBOA CAPITAL%%', '%%CREST CAPITAL%%',
  '%%KEYSTONE FUNDING%%', '%%FAASTRAK%%', '%%HALO CAPITAL%%',
  '%%SNAP ADVANCES%%', '%%JET CAPITAL%%', '%%SHARPSHOOTER FUNDING%%', '%%MULLIGAN FUNDING%%', '%%GREENBOX CAPITAL%%')
   THEN abs(amount) ELSE 0 END) > 200
       THEN 1 ELSE 0 END AS  business_loan,
        CASE WHEN SUM(CASE WHEN type='debit' and OWNED_BY ='person' and
UPPER(DESCRIPTION) ilike any ('%%WEWORK%%', '%%REGUS%%', '%%INDUSTRIOUS%%', '%%SPACES%%',
                              '%%SERENDIPITY LABS%%', '%%KNOTEL%%', '%%IMPACT HUB%%',
                              '%%TECHSPACE%%', '%%WORKBAR%%', '%%THE YARD%%', '%%NEUEHOUSE%%',
                              '%%CONVENE%%', '%%PREMIER WORKSPACES%%', '%%OFFICE EVOLUTION%%',
                              '%%COMMONGROUNDS WORKPLACE%%', '%%THE WING%%', '%%CARR WORKPLACES%%',
                              '%%THE FARM SOHO%%', '%%VENTURE X%%', '%%HERA HUB%%', '%%DESKPASS%%',
                              '%%MAKEOFFICES%%', '%%SECOND HOME%%', '%%LIQUIDSPACE%%', '%%THE RIVETER%%',
                              '%%PROXIMITY SPACE%%', '%%COLAB COWORKING%%', '%%BOND COLLECTIVE%%',
                              '%%ENERSPACE COWORKING%%', '%%GATHER%%', '%%HATCH COWORKING%%',
                              '%%BUROHQ%%', '%%NOVEL COWORKING%%', '%%ALLEY%%', '%%THE SATELLITE%%',
                              '%%GREEN SPACES%%', '%%OFFICE NOMADS%%', '%%CATAPULT%%', '%%CENDYN SPACES%%',
                              '%%UNION COWORK%%', '%%NEXCORE%%', '%%GRIND%%', '%%HERA HUB%%', '%%PILOT%%',
                              '%%THE PAD%%', '%%BLANKSPACES%%', '%%THE PORT WORKSPACES%%', '%%SERENDIPITY LABS%%',
                              '%%HERA HUB%%', '%%ENERSPACE COWORKING%%', '%%BUROHQ%%')
                      THEN 1 ELSE 0 END) >0 AND
             SUM(CASE WHEN type='debit' and OWNED_BY ='person' and
UPPER(DESCRIPTION) ilike any ('%%WEWORK%%', '%%REGUS%%', '%%INDUSTRIOUS%%', '%%SPACES%%',
                              '%%SERENDIPITY LABS%%', '%%KNOTEL%%', '%%IMPACT HUB%%',
                              '%%TECHSPACE%%', '%%WORKBAR%%', '%%THE YARD%%', '%%NEUEHOUSE%%',
                              '%%CONVENE%%', '%%PREMIER WORKSPACES%%', '%%OFFICE EVOLUTION%%',
                              '%%COMMONGROUNDS WORKPLACE%%', '%%THE WING%%', '%%CARR WORKPLACES%%',
                              '%%THE FARM SOHO%%', '%%VENTURE X%%', '%%HERA HUB%%', '%%DESKPASS%%',
                              '%%MAKEOFFICES%%', '%%SECOND HOME%%', '%%LIQUIDSPACE%%', '%%THE RIVETER%%',
                              '%%PROXIMITY SPACE%%', '%%COLAB COWORKING%%', '%%BOND COLLECTIVE%%',
                              '%%ENERSPACE COWORKING%%', '%%GATHER%%', '%%HATCH COWORKING%%',
                              '%%BUROHQ%%', '%%NOVEL COWORKING%%', '%%ALLEY%%', '%%THE SATELLITE%%',
                              '%%GREEN SPACES%%', '%%OFFICE NOMADS%%', '%%CATAPULT%%', '%%CENDYN SPACES%%',
                              '%%UNION COWORK%%', '%%NEXCORE%%', '%%GRIND%%', '%%HERA HUB%%', '%%PILOT%%',
                              '%%THE PAD%%', '%%BLANKSPACES%%', '%%THE PORT WORKSPACES%%', '%%SERENDIPITY LABS%%',
                              '%%HERA HUB%%', '%%ENERSPACE COWORKING%%', '%%BUROHQ%%')
   THEN abs(amount) ELSE 0 END) > 1000
       THEN 1 ELSE 0 END AS  rental_co_work,
    CASE WHEN SUM(CASE WHEN type='credit' and OWNED_BY ='person' and categories ilike '%Shops%'  THEN 1 ELSE 0 END) >= 10 AND
                      SUM(CASE WHEN type='debit' and OWNED_BY ='person' and  categories ilike '%Shops%' THEN abs(amount) ELSE 0 END) > 2000
                      THEN 1 ELSE 0 END AS  shops_money,
       (flag_payroll + flag_rent + online_platform_deposit + business_like_payment+ business_loan+rental_co_work+shops_money) AS sum_flags
FROM prod_db.adhoc.lending_dda_txns_w_owned_20230624
where owned_by='person'
GROUP BY 1)
where sum_flags>0
),
buss_tag as (
select distinct a.business_id, a.external_account_id,
case when (a.owned_by is null or a.owned_by='' or a.owned_by='business') or (a.owned_by='person' and (b.business_flag='business')) then 1 else 0 end as business_tag
from txn_base a
left join ext_acct_buss b
on a.external_account_id = b.external_account_id
),
person_txns as (
select distinct a.business_id, a.external_account_id, a.transaction_id,
case when 
(categories ilike '%Community%' and type='debit') or
(categories ilike '%Food and Drink%' and type='debit') or
(categories ilike '%Healthcare%' and type='debit') or
(categories ilike '%Recreation%' and type='debit') or
(categories ilike '%Payment,Rent%' and type='debit') or
(categories ilike '%Service%' and type='debit') or
(categories ilike '%Shops%' and type='debit') or
(categories ilike '%Travel%' and type='debit') or
(categories ilike '%Payroll%' and type='credit') or
(categories ilike '%Loan%') or
(categories ilike '%Tax%') or
(categories ilike '%Interest%') or
(categories ilike '%Transfer,Keep the Change Savings Program%') or
(categories ilike '%Transfer%' and abs(amount)<1000) or
(categories ilike '%Service,Financial,Loans and Mortgages%') or
(categories ilike '%Service,Financial,Taxes%') or
(categories ilike '%Insurance%') or
(categories ilike '%Transfer,Save As You Go%')
then 0 else 1 end as txn_inclusion_flag
from txn_base a
join buss_tag b
on a.business_id = b.business_id
and a.external_account_id = b.external_account_id
where b.business_tag = 0
),
selected_txns as (
select a.*
from txn_base a
join person_txns b
on a.external_account_id = b.external_account_id
and a.transaction_id = b.transaction_id
where txn_inclusion_flag=1
union
select a.* 
from txn_base a
join buss_tag b
on a.external_account_id = b.external_account_id
where business_tag=1
)
select * from selected_txns
order by business_id, external_account_id, date
""")
filtered_txns.shape, filtered_txns.business_id.nunique(), filtered_txns.transaction_id.nunique()

((200514, 21), 645, 200513)

In [ ]:
tagged_txns.shape, filtered_txns.shape

((242282, 21), (200514, 21))

In [ ]:
tagged_txns.groupby('owned_by').size(), filtered_txns.groupby('owned_by').size()

(owned_by
               1895
 business     84668
 person      155719
 dtype: int64,
 owned_by
               1895
 business     84668
 person      113951
 dtype: int64)

#### writing the final data

In [ ]:
from sqlalchemy.types import NVARCHAR

filtered_txns.to_sql(name='lending_dda_txns_w_owned_flag_final_20230625',
                 con=q.engine, 
                 schema='prod_db.adhoc',
                 if_exists='append', 
                 index=False, 
                 chunksize=16000, 
                 method='multi',
                 dtype={col_name: NVARCHAR for col_name in filtered_txns})

200514